In [1]:
import statsapi as mlb
import pandas as pd
import sqlalchemy
import config
import csv
from os import walk
import re
from datetime import datetime
import functions
import sql_alch_schema
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String,DateTime,Date,Time,Boolean,Float
from sqlalchemy import ForeignKey,and_
from sqlalchemy.orm import relationship


_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.user}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )
## Create the engine 
db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True,})

Base = sql_alch_schema.Base

Game = sql_alch_schema.Game
Play = sql_alch_schema.Play
Team = sql_alch_schema.Team
GameTeamLink = sql_alch_schema.GameTeamLink
Person = sql_alch_schema.Person
PersonPlayLink=sql_alch_schema.PersonPlayLink



from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()

In this notebook, I want to break up the first part of my datapipeline in 3 parts, saving them to CSVs at each step. The 3 steps are:
    - sql query to get cumulative stat totals by player by game 
    - turn those cumulative totals into averages like obp, slg, and ba 
    - aggregate player averages by team

I want these functions to be combatible with an sklearn pipeline so I can adjust how the informations is interpreted by the classifier.

First step is to collect cumulated totals by player by game and save to csv

In [2]:
game=session.query(Game).filter(Game.type=='R').first()

In [13]:
[v.id for v in game.game_players(session)['home']]

[595014,
 656305,
 641941,
 621471,
 459964,
 519295,
 622194,
 621566,
 476704,
 640461,
 488748,
 518960,
 501822,
 433586,
 608665,
 605254,
 640455,
 501981,
 596043,
 572039,
 521230,
 543056,
 433589,
 592811,
 543760]